In [2]:
# 01 — Data Ingestion & Quality Checks (Daily Market Data)

**Project:** value-at-risk-backtesting-suite  
**Goal:** Download daily market data, run quality checks, and create baseline features (log returns) for risk modeling (VaR, GARCH, ML).

**Data source:** Stooq (via `pandas_datareader`)  
**Asset universe (MVP):** SPY (S&P 500 ETF)

SyntaxError: invalid syntax (587485538.py, line 3)

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pandas_datareader import data as pdr

pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 120)

START = "2010-01-01"
TICKERS = ["spy.us"]  # puedes sumar "qqq.us" después


/Users/maximilianoyanez/Documents/value-at-risk-backtesting-suite/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
def load_stooq_daily(symbol: str, start: str = START) -> pd.DataFrame:
    """
    Download daily OHLCV data from Stooq via pandas_datareader.
    Returns a DataFrame indexed by date in ascending order.
    """
    df = pdr.DataReader(symbol, "stooq")
    df = df.sort_index()  # Stooq a veces viene descendente
    df = df.loc[df.index >= pd.to_datetime(start)]
    df.columns = [c.lower() for c in df.columns]  # open/high/low/close/volume
    df["ticker"] = symbol
    return df

data = pd.concat([load_stooq_daily(t) for t in TICKERS], axis=0).reset_index().rename(columns={"index": "date"})
data.head()
